In [9]:
import gensim
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamodel import LdaModel
import os
import time
import pandas as pd
%env JOBLIB_TEMP_FOLDER= 'E:/sp2023stock/TopicModeling/tmp'


env: JOBLIB_TEMP_FOLDER='E:/sp2023stock/TopicModeling/tmp'


#### Modeling

In [5]:
num = 100

main = os.getcwd()
#in
#model_path_asset = main + "/../model/ModelingAssets/CorpusDict"
model_path_asset_1mill = main + "/../model/ModelingAssets/1mill/CorpusDict"

#out
model_path_state = main + "/../model/ModelingState"
output_result = main + "/../result/modelOutput"


model_path_asset_spenum = model_path_asset_1mill+str(num)
model_path_asset_spenum

'E:\\sp2023stock\\TopicModeling\\src/../model/ModelingAssets/1mill/CorpusDict100'

In [6]:

print("Dataset loading  dictionary - corpus - json")
# Load the dictionary and corpus
dictionary = Dictionary.load(f'{model_path_asset_spenum}/dictionary.gensim')
corpus = MmCorpus(f'{model_path_asset_spenum}/corpus.mm')
# json_file_path = f'{model_path_asset_spenum}/dataset_{num}.json'

# with open(json_file_path, 'r', encoding='utf-8') as json_file:
#     dataset = json.load(json_file)
    
print(f"Dataset loaded - dictionary {len(dictionary)} - corpus {len(corpus)} - json")


Dataset loading  dictionary - corpus - json
Dataset loaded - dictionary 225206 - corpus 1156987 - json


In [7]:
num_topics = 400
total_iterations = 2000
save_interval = 100

# Initialize LDA model

print(f"Running the model using corpus: {len(corpus)}, iterations: {total_iterations}, number of topic: {num_topics}")

lda = gensim.models.ldamulticore.LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    iterations= total_iterations
    
)

print("Finishing")
print("\n")
print("Saving")

filename = os.path.join(model_path_state, f'lda_model_n{num_topics}_iterations{total_iterations}_size100.gensim')
lda.save(filename)
print(f"Saved model at pass number {total_iterations} to {filename}")



Running the model using corpus: 1156987, iterations: 2000, number of topic: 400
Finishing


Saving


NameError: name 'model_path_state' is not defined

In [9]:
filename = os.path.join(model_path_state, f'lda_model_n{num_topics}_iterations{total_iterations}_size100.gensim')
lda.save(filename)
print(f"Saved model at pass number {total_iterations} to {filename}")

Saved model at pass number 2000 to E:\sp2023stock\TopicModeling\src/../model/ModelingState\lda_model_n400_iterations2000_size100.gensim


#### Loading (optional)

In [11]:
filename = os.path.join(model_path_state, f'lda_model_n400_iterations2000_size100.gensim')
lda = LdaModel.load(filename)
print(f"Load model Completed")

Load model Completed


#### Inferanceing

###### Print Distribution of each articles

In [25]:
# Get topic distribution for each document
doc_topic_dists = [lda.get_document_topics(item, minimum_probability=0) for item in corpus]

# Convert the topic distributions into a structured format
doc_topics = []
for doc_dist in doc_topic_dists:
    doc_topics.append([prob for _, prob in doc_dist])

# Create a DataFrame
doc_topics_df = pd.DataFrame(doc_topics, columns=[f"Topic_{i}" for i in range(num_topics)])

# # If you have a list of article IDs, you can insert them too
# doc_topics_df.insert(0, 'Article_ID', processed_article_ids)
doc_topic_out = os.path.join(output_result, 'document_topics.csv') 
doc_topics_df.to_csv(doc_topic_out, index=False)

print(f"Topic distribution for each article is completely saved at {doc_topic_out}")

###### Print Words

In [13]:
# Initialize a list to store the topics data
topics_data = []
num_topics = 400
num_showwords = 100
print(f"Total of words in each topic are{num_showwords}") 

for i in range(num_topics):
    words = lda.show_topic(i, num_showwords)
    topic_words = [word[0] for word in words]
    topics_data.append(topic_words)

# Create a DataFrame from the topics data
df_topics = pd.DataFrame(topics_data, index=[f"Topic {i}" for i in range(num_topics)], columns=[f"Word {i+1}" for i in range(num_showwords)])

# Save the DataFrame to a CSV file
csv_file_path = os.path.join(output_result, 'word_topics.csv')
df_topics.to_csv(csv_file_path, index=True)

print(f"Words for each topic is completely saved at {csv_file_path}")

Discovered Topics:
Topics saved to E:\sp2023stock\TopicModeling\src/../result/modelOutput\word_topics.csv


#### Result

In [41]:
df = pd.read_csv(doc_topic_out)
len(df)
df

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,...,Topic_390,Topic_391,Topic_392,Topic_393,Topic_394,Topic_395,Topic_396,Topic_397,Topic_398,Topic_399
0,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,...,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104
1,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,...,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071
2,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,...,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125
3,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,...,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042
4,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,...,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038,0.000038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156982,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,...,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076
1156983,0.000031,0.000031,0.057179,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,...,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031
1156984,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,...,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.386651,0.000046
1156985,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,...,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026


In [43]:
masterFileasd = 'E:/sp2023stock/TopicModeling/data/ModelingDataset/Test/V.2/MasterDesctiption.csv'
masterdf = pd.read_csv(masterFileasd, nrows=5)
masterdf

,ID,Agency,Date,Links,Title,Status
0,13,prachachat,09/06/2017 09:49:48,https://www.prachachat.net/finance/news-13,จองพื้นที่มอเตอร์เอ็กซ์โปคึก เชื่อยอดขายพุ่ง,True
1,20,prachachat,09/06/2017 10:12:16,https://www.prachachat.net/finance/news-20,ทอท.ปรับดอนเมืองเฟส 3 เพิ่มระบบ APM-ชงบอร์ด พ....,True
2,21,prachachat,09/06/2017 10:15:58,https://www.prachachat.net/finance/news-21,"""คีรี"" รีเทิร์นรอบ 25 ปีลงทุนรถไฟฟ้า เราเคยบาด...",True
3,22,prachachat,09/06/2017 10:16:00,https://www.prachachat.net/finance/news-22,อัดขายบ้านสายสีม่วงลุ้น Q2 เชื่อมแน่สถานีเตาปู...,True
4,25,prachachat,09/06/2017 10:16:04,https://www.prachachat.net/finance/news-25,ธปท.ผ่อนคุมแลกเงิน-เพิ่มผู้เล่น เอื้อธุรกิจ-รา...,True


In [49]:
dataFile = 'E:/sp2023stock/TopicModeling/data/ModelingDataset/Test/V.2/dataset.csv'
datadf = pd.read_csv(dataFile)
datadf = datadf.drop('Article', axis=1)
datadf.to_csv('E:/sp2023stock/TopicModeling/data/ModelingDataset/Test/V.2/dateonly.csv', index=False)

In [62]:
dateonly_p = 'E:/sp2023stock/TopicModeling/data/ModelingDataset/Test/V.2/dateonly.csv'
dateonly = pd.read_csv(dateonly_p)
len(dateonly)

1156987

In [63]:
dateonly

,ID,Agency,Date
0,13,prachachat,2017-06-09
1,20,prachachat,2017-06-09
2,21,prachachat,2017-06-09
3,22,prachachat,2017-06-09
4,25,prachachat,2017-06-09
...,...,...,...
1156982,1042510,dailynews,2022-05-14
1156983,1042584,dailynews,2022-05-12
1156984,1042589,dailynews,2022-05-12
1156985,1042592,dailynews,2022-05-12


In [64]:
# Merging
df1 = pd.read_csv(doc_topic_out) #top 3
df2 = pd.read_csv(dateonly) # master

assert len(df1) == len(df2)

for column in df2.columns:
    df1[column] = df2[column].values


TypeError: argument of type 'method' is not iterable